In [65]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

In [57]:
df=pd.read_csv('Ramayan.txt',sep='\t',header=None,names=["Texts"])

In [58]:
texts=[]
for i in df['Texts']:
    a=i.split(".")
    for text in a:
        texts.append(text)

In [59]:
df.drop(columns='Texts',inplace=True)

In [63]:
new_col=pd.DataFrame(texts,columns=['TEXTS'])
new_col

,TEXTS
0,On the banks of the Sarayu River stood the bea...
1,In the city there were magnificent palaces de...
2,Spires of great temples rose above the city a...
3,"For protection, the city was surrounded by a ..."
4,The people of Ayodhya were peace-loving and h...
...,...
889,
890,Rama entered the palace and Sita bowed at his ...
891,He felt both love and sorrow for his wife
892,"""We shall return to Ayodhya,"" he said"


In [131]:
new_col['TEXTS'][0]

'On the banks of the Sarayu River stood the beautiful city of Ayodhya, the capital of Kosala'

In [66]:
tokenizer=Tokenizer()

In [67]:
tokenizer.fit_on_texts(new_col['TEXTS'])

In [89]:
len(tokenizer.word_index)

1382

In [70]:
input_sequences=[]
for sentence in new_col['TEXTS']:
    token_list=tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [71]:
input_sequences

[[31, 1],
 [31, 1, 668],
 [31, 1, 668, 6],
 [31, 1, 668, 6, 1],
 [31, 1, 668, 6, 1, 445],
 [31, 1, 668, 6, 1, 445, 281],
 [31, 1, 668, 6, 1, 445, 281, 145],
 [31, 1, 668, 6, 1, 445, 281, 145, 1],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81, 68],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81, 68, 6],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81, 68, 6, 63],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81, 68, 6, 63, 1],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81, 68, 6, 63, 1, 669],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81, 68, 6, 63, 1, 669, 6],
 [31, 1, 668, 6, 1, 445, 281, 145, 1, 81, 68, 6, 63, 1, 669, 6, 348],
 [13, 1],
 [13, 1, 68],
 [13, 1, 68, 87],
 [13, 1, 68, 87, 45],
 [13, 1, 68, 87, 45, 446],
 [13, 1, 68, 87, 45, 446, 670],
 [13, 1, 68, 87, 45, 446, 670, 671],
 [13, 1, 68, 87, 45, 446, 670, 671, 22],
 [13, 1, 68, 87, 45, 446, 670, 671, 22, 672],
 [13, 1, 68, 87, 45, 446, 670, 671, 22, 672, 447],
 [673, 6],
 [673, 6, 48],
 [673, 6, 48, 674],
 [673, 6, 48

In [76]:
total_length=max([len(a) for a in input_sequences])

In [79]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequence=pad_sequences(input_sequences,maxlen=total_length,padding='pre')

In [80]:
padded_input_sequence

array([[  0,   0,   0, ...,   0,  31,   1],
       [  0,   0,   0, ...,  31,   1, 668],
       [  0,   0,   0, ...,   1, 668,   6],
       ...,
       [  0,   0,   0, ..., 109,   2,  63],
       [  0,   0,   0, ...,   2,  63,   5],
       [  0,   0,   0, ...,  63,   5,  16]])

In [82]:
X = padded_input_sequence[:,:-1]

In [90]:
y=padded_input_sequence[:,-1]
y

array([  1, 668,   6, ...,  63,   5,  16])

In [84]:
X.shape

(6361, 33)

In [85]:
y.shape

(6361,)

In [92]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y,num_classes=1383)

In [94]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [115]:
model_new = Sequential()
model_new.add(Embedding(1383, 100, input_length=33))
model_new.add(LSTM(150))  # Set return_sequences=True here# Leave this as it is, since it's the last LSTM layer
model_new.add(Dense(1383, activation='softmax'))


In [116]:
model_new.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [104]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [117]:
model_new.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [137]:
model_new.fit(X,y,epochs=100)

Epoch 1/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8826 - loss: 0.3525
Epoch 2/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.8862 - loss: 0.3441
Epoch 3/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8838 - loss: 0.3451
Epoch 4/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8840 - loss: 0.3377
Epoch 5/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.8886 - loss: 0.3342
Epoch 6/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8757 - loss: 0.3697
Epoch 7/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8875 - loss: 0.3305
Epoch 8/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.8869 - loss: 0.3393
Epoch 9/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.8716 - loss: 0.3664
Epoch 10/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.8792 - loss: 0.3361
Epoch 11/100
199/199 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.8867 - loss: 0.3171
Epoch 12/100
199/199 ━━━━━━━━━

In [138]:
import time
text = "Rama and Sita"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=33, padding='pre')
  # predict
  pos = np.argmax(model_new.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Rama and Sita continued
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Rama and Sita continued to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Rama and Sita continued to serve
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Rama and Sita continued to serve their
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Rama and Sita continued to serve their parents
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Rama and Sita continued to serve their parents and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Rama and Sita continued to serve their parents and delight
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Rama and Sita continued to serve their parents and delight the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Rama and Sita continued to serve their parents and delight the holy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Rama and Sita continued to serve their parents and delight the holy ones


In [139]:
import time
text = "sita and rama"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=34, padding='pre')
  # predict
  pos = np.argmax(model_new.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
sita and rama were
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
sita and rama were the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
sita and rama were the perfect
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
sita and rama were the perfect husband
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
sita and rama were the perfect husband and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
sita and rama were the perfect husband and wife
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
sita and rama were the perfect husband and wife he
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
sita and rama were the perfect husband and wife he is
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
sita and rama were the perfect husband and wife he is the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
sita and rama were the perfect husband and wife he is the man


In [140]:
import time
text = "Bharat was the brother of"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=34, padding='pre')
  # predict
  pos = np.argmax(model_new.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Bharat was the brother of rama
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Bharat was the brother of rama hanuman
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Bharat was the brother of rama hanuman and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Bharat was the brother of rama hanuman and marched
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Bharat was the brother of rama hanuman and marched into
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Bharat was the brother of rama hanuman and marched into the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bharat was the brother of rama hanuman and marched into the forest
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Bharat was the brother of rama hanuman and marched into the forest of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bharat was the brother of rama hanuman and marched into the forest of lanka
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Bharat was the brother of rama hanuman and marched into the forest of lanka ablaze
